In [4]:
import os
import sys
from subprocess import check_call

check_call([sys.executable, '-m' 'pip', 'install', '--user', '-U', 'pip'])

0

In [59]:
#check_call([sys.executable, '-mpip', 'install', '--user', '-U', 'alerce==1.2.0'])
#check_call([sys.executable, '-mpip', 'install', '--user', '-U', 'astro-ghost==0.2.3'])
check_call([sys.executable, '-mpip', 'install', '--user', '-U', 'opencv-python==4.7.0.72'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 7.4 MB/s eta 0:00:001


0

In [60]:
# LAISS Random Forest Classifier for Transient (Supernova) Anomaly Detection
# Written by Patrick Aleo

# Import all necessary packages
import antares.devkit as dk; dk.init()

from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames

import os
import sys
import time

from alerce.core import Alerce
alerce = Alerce()

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
from datetime import datetime
import tempfile

import astro_ghost
# from astro_ghost.PS1QueryFunctions import getAllPostageStamps
# from astro_ghost.TNSQueryFunctions import getTNSSpectra
# from astro_ghost.NEDQueryFunctions import getNEDSpectra
from astro_ghost.ghostHelperFunctions import getTransientHosts, getGHOST

from astropy import units as u
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames

import sklearn.ensemble
from sklearn.ensemble import IsolationForest

import warnings
warnings.filterwarnings("ignore")

# Define some useful functions and feature lists and transformations

# From 106 available features from Kostya's lc_feature_extractor, use the 82 from SNAD Miner paper 
# R and g bands
feature_names_r_g = ['feature_amplitude_magn_r', 
                         'feature_anderson_darling_normal_magn_r',
                         'feature_beyond_1_std_magn_r', 
                         'feature_beyond_2_std_magn_r',
                         'feature_cusum_magn_r', 
                         #'feature_eta_e_magn_r',
                         'feature_inter_percentile_range_2_magn_r',
                         'feature_inter_percentile_range_10_magn_r',
                         'feature_inter_percentile_range_25_magn_r', 
                         'feature_kurtosis_magn_r',
                         'feature_linear_fit_slope_magn_r',
                         'feature_linear_fit_slope_sigma_magn_r',
                         #'feature_linear_fit_reduced_chi2_magn_r', 
                         #'feature_linear_trend_magn_r', # cadence removal
                         #'feature_linear_trend_sigma_magn_r',  # cadence removal
                         'feature_magnitude_percentage_ratio_40_5_magn_r',
                         'feature_magnitude_percentage_ratio_20_5_magn_r',
                         #'feature_maximum_slope_magn_r',
                         'feature_mean_magn_r',
                         'feature_median_absolute_deviation_magn_r',
                         'feature_percent_amplitude_magn_r',
                         'feature_median_buffer_range_percentage_10_magn_r',
                         'feature_median_buffer_range_percentage_20_magn_r',
                         'feature_percent_difference_magnitude_percentile_5_magn_r',
                         'feature_percent_difference_magnitude_percentile_10_magn_r',
                         #'feature_period_0_magn_r',  # should be negated
                         #'feature_period_s_to_n_0_magn_r', # cadence removal
                         #'feature_period_1_magn_r', 
                         #'feature_period_s_to_n_1_magn_r', # cadence removal
                         #'feature_period_2_magn_r', 
                         #'feature_period_s_to_n_2_magn_r', # cadence removal
                         #'feature_period_3_magn_r', 
                         #'feature_period_s_to_n_3_magn_r', # cadence removal
                         #'feature_period_4_magn_r', 
                         #'feature_period_s_to_n_4_magn_r', # cadence removal
                         #'feature_periodogram_amplitude_magn_r',
                         #'feature_periodogram_beyond_2_std_magn_r',  # cadence removal
                         #'feature_periodogram_beyond_3_std_magn_r',  # cadence removal
                         #'feature_periodogram_standard_deviation_magn_r',   # cadence removal
                         #'feature_chi2_magn_r',
                         'feature_skew_magn_r', 
                         'feature_standard_deviation_magn_r',
                         'feature_stetson_k_magn_r',
                         'feature_weighted_mean_magn_r',
                         'feature_anderson_darling_normal_flux_r', 
                         'feature_cusum_flux_r',
                         #'feature_eta_e_flux_r', 
                         'feature_excess_variance_flux_r',
                         'feature_kurtosis_flux_r', 
                         'feature_mean_variance_flux_r',
                         #'feature_chi2_flux_r', 
                         'feature_skew_flux_r',
                         'feature_stetson_k_flux_r',

                         'feature_amplitude_magn_g', 
                         'feature_anderson_darling_normal_magn_g',
                         'feature_beyond_1_std_magn_g', 
                         'feature_beyond_2_std_magn_g',
                         'feature_cusum_magn_g', 
                         #'feature_eta_e_magn_g',
                         'feature_inter_percentile_range_2_magn_g',
                         'feature_inter_percentile_range_10_magn_g',
                         'feature_inter_percentile_range_25_magn_g', 
                         'feature_kurtosis_magn_g',
                         'feature_linear_fit_slope_magn_g',
                         'feature_linear_fit_slope_sigma_magn_g',
                         #'feature_linear_fit_reduced_chi2_magn_g', 
                         #'feature_linear_trend_magn_g', # cadence removal
                         #'feature_linear_trend_sigma_magn_g',  # cadence removal
                         'feature_magnitude_percentage_ratio_40_5_magn_g',
                         'feature_magnitude_percentage_ratio_20_5_magn_g',
                         #'feature_maximum_slope_magn_g', 
                         'feature_mean_magn_g',
                         'feature_median_absolute_deviation_magn_g',
                         'feature_median_buffer_range_percentage_10_magn_g',
                         'feature_median_buffer_range_percentage_20_magn_g',
                         'feature_percent_amplitude_magn_g',
                         'feature_percent_difference_magnitude_percentile_5_magn_g',
                         'feature_percent_difference_magnitude_percentile_10_magn_g',
                         #'feature_period_0_magn_g',  # should be negated
                         #'feature_period_s_to_n_0_magn_g', # cadence removal
                         #'feature_period_1_magn_g', 
                         #'feature_period_s_to_n_1_magn_g', # cadence removal
                         #'feature_period_2_magn_g', 
                         #'feature_period_s_to_n_2_magn_g', # cadence removal
                         #'feature_period_3_magn_g', 
                         #'feature_period_s_to_n_3_magn_g', # cadence removal
                         #'feature_period_4_magn_g', 
                         #'feature_period_s_to_n_4_magn_g', # cadence removal
                         #'feature_periodogram_amplitude_magn_g',
                         #'feature_periodogram_beyond_2_std_magn_g',  # cadence removal
                         #'feature_periodogram_beyond_3_std_magn_g', # cadence removal
                         #'feature_periodogram_standard_deviation_magn_g',  # cadence removal
                         #'feature_chi2_magn_g',
                         'feature_skew_magn_g', 
                         'feature_standard_deviation_magn_g',
                         'feature_stetson_k_magn_g', 
                         'feature_weighted_mean_magn_g',
                         'feature_anderson_darling_normal_flux_g', 
                         'feature_cusum_flux_g',
                         #'feature_eta_e_flux_g', 
                         'feature_excess_variance_flux_g',
                         'feature_kurtosis_flux_g', 
                         'feature_mean_variance_flux_g',
                         #'feature_chi2_flux_g', 
                         'feature_skew_flux_g',
                         'feature_stetson_k_flux_g'] 

def get_alerce_class(output):
    """
    Given the top 'output' probabilities from Alerce's classifiers (lc_classifier, hierarchical_rf, stamp_classifier),
    return the best class (the one displayed on the webpage at https://alerce.online/object/, typically lc_classifier). 
    If not classified, return 'Not_classified'.
    """
    top_class = None
    for item in output:
        if item['ranking'] == 1:
            if top_class is None or \
               (item['classifier_name'] == 'lc_classifier' and \
                item['classifier_version'] == 'hierarchical_rf_1.1.0') or \
               (item['classifier_name'] == 'stamp_classifier' and \
                item['classifier_version'] == 'stamp_classifier_1.0.4' and \
                top_class['classifier_version'] != 'hierarchical_rf_1.1.0') or \
               (item['classifier_name'] == 'stamp_classifier' and \
                item['classifier_version'] == 'stamp_classifier_1.0.0' and \
                top_class['classifier_version'] not in ['hierarchical_rf_1.1.0', 'stamp_classifier_1.0.4']):
                top_class = item
    return top_class['class_name'] if top_class is not None else "Not_classified"

Jaeger tracer already initialized, skipping


2023-06-27 14:18:48,768 - WARNING MainThread settings.py:setup_prometheus:124 - Prometheus failed to start with [Errno 98] Address already in use
Testing loading a random Locus with `dk.get_locus()`...
2023-06-27 14:18:50,866 - INFO MainThread settings.py:cassandra_session_factory:84 - Establishing connection to Cassandra
2023-06-27 14:18:52,024 - INFO MainThread settings.py:cassandra_session_factory:88 - Connection to Cassandra established
2023-06-27 14:18:52,026 - INFO MainThread settings.py:cassandra_session_factory:84 - Establishing connection to Cassandra
2023-06-27 14:18:53,283 - INFO MainThread settings.py:cassandra_session_factory:88 - Connection to Cassandra established
2023-06-27 14:18:53,284 - INFO MainThread settings.py:cassandra_session_factory:84 - Establishing connection to Cassandra
2023-06-27 14:18:54,454 - INFO MainThread settings.py:cassandra_session_factory:88 - Connection to Cassandra established

ANTARES v2.4.2 DevKit is ready!
Website: https://antares.noirlab.edu

In [67]:
class LAISS_RFC_AD_filter(dk.Filter):
    NAME = 'LAISS_RFC_AD_filter'
    ERROR_SLACK_CHANNEL = 'U0135TMN9UK'  # Patrick Aleo Slack member ID
    REQUIRES_FILES = [
    #    './paleo2_106IFmodel_472727objs_nest=1000_cont=0.001_transf.pkl', #106 feature Isolation Forest model
    #    './antares_norm.py' #py file for transforming ZTF alert feature values
    ]
    REQUIRED_LOCUS_PROPERTIES = [
        'ztf_object_id',
    ]
    REQUIRED_ALERT_PROPERTIES = [
    ]
    INPUT_TAGS = [
        'lc_feature_extractor'
    ]
    OUTPUT_LOCUS_PROPERTIES = [
          {
          'name': 'LAISS_RFC_anomaly_score',
          'type': 'float',
          'description': 'Calculated anomaly score. From RFC model, objects are tagged as anomalous if "Other" (anomaly) classification is >0.5 at any point during light curve evolution. If none calculated, set to TBD',
          },
    ]
    OUTPUT_ALERT_PROPERTIES = [
    ]
    OUTPUT_TAGS = [
        {
          'name': 'LAISS_RFC_AD_filter', 
          'description': 'The Light Curve AI Similarity Search (LAISS; Aleo et al in prep) random forest classifier anomaly detection filter aims to tag rare, exotic, and unusual transients (supernovae). It is an random forest classifier algorithm trained on upsampled (balanced classes) spectroscopic SNe using extracted light curve and host galaxy features.',
        },
        {
          'name': 'RFC_AD_status',
          'type': 'str',
          'description': 'Status of anomalous loci. Normal if non-anomalous; Other if anomalous; TBD if to be determined once lc_feature_extractor is run in both passbands',
        },
    ]

    
    def setup(self):
        """
    #    ANTARES will call this function once at the beginning of each night
    #    when filters are loaded.
    #    """
#         # Load 106 feature IF model at beginning of the night and store it. 
#         #self.IF_model = self.files['./paleo2_106IFmodel_472727objs_nest=1000_cont=0.001_transf.pkl']
#         with open('./paleo2_106IFmodel_472727objs_nest=1000_cont=0.001_transf.pkl', 'rb') as f: #not imported right way
#             self.clf = pickle.load(f)
        
#         # Return the model to use in run(self, locus)
#         return self.clf
        return self

    def run(self, locus):
        """
        Send alerts to stream 'LAISS_RFC_AD_filter' if alert was received from 'lc_feature_extractor' filter,
        assigned an anomaly score from trained RFC model (requires both R, g bands),
        and passes strict masking requirement to filter out unwanted periodic objects to maximise anomalous,
        transient events. 
        """
        
        # Required: if the loci has passed through the 'lc_feature_extractor' filter
        if 'lc_feature_extractor' in locus.tags:
            
            # from locus info, get ra,dec and cut on objs in galactic plane
            coord_testset = SkyCoord(locus.ra, 
                                     locus.dec, 
                                     frame="icrs", 
                                     unit='deg')
            galcoord_testset = coord_testset.galactic
            # cut on anything in galactic plane
            if galcoord_testset.b.degree < 15:
                print("In galactic plane. Don't use. Skip!")
                return

            # From 106 available features from Kostya's lc_feature_extractor, use the 82 from SNAD Miner paper 
            # R and g bands
            feature_names_r_g = ['feature_amplitude_magn_r', 
                         'feature_anderson_darling_normal_magn_r',
                         'feature_beyond_1_std_magn_r', 
                         'feature_beyond_2_std_magn_r',
                         'feature_cusum_magn_r', 
                         #'feature_eta_e_magn_r',
                         'feature_inter_percentile_range_2_magn_r',
                         'feature_inter_percentile_range_10_magn_r',
                         'feature_inter_percentile_range_25_magn_r', 
                         'feature_kurtosis_magn_r',
                         'feature_linear_fit_slope_magn_r',
                         'feature_linear_fit_slope_sigma_magn_r',
                         #'feature_linear_fit_reduced_chi2_magn_r', 
                         #'feature_linear_trend_magn_r', # cadence removal
                         #'feature_linear_trend_sigma_magn_r',  # cadence removal
                         'feature_magnitude_percentage_ratio_40_5_magn_r',
                         'feature_magnitude_percentage_ratio_20_5_magn_r',
                         #'feature_maximum_slope_magn_r',
                         'feature_mean_magn_r',
                         'feature_median_absolute_deviation_magn_r',
                         'feature_percent_amplitude_magn_r',
                         'feature_median_buffer_range_percentage_10_magn_r',
                         'feature_median_buffer_range_percentage_20_magn_r',
                         'feature_percent_difference_magnitude_percentile_5_magn_r',
                         'feature_percent_difference_magnitude_percentile_10_magn_r',
                         #'feature_period_0_magn_r',  # should be negated
                         #'feature_period_s_to_n_0_magn_r', # cadence removal
                         #'feature_period_1_magn_r', 
                         #'feature_period_s_to_n_1_magn_r', # cadence removal
                         #'feature_period_2_magn_r', 
                         #'feature_period_s_to_n_2_magn_r', # cadence removal
                         #'feature_period_3_magn_r', 
                         #'feature_period_s_to_n_3_magn_r', # cadence removal
                         #'feature_period_4_magn_r', 
                         #'feature_period_s_to_n_4_magn_r', # cadence removal
                         #'feature_periodogram_amplitude_magn_r',
                         #'feature_periodogram_beyond_2_std_magn_r',  # cadence removal
                         #'feature_periodogram_beyond_3_std_magn_r',  # cadence removal
                         #'feature_periodogram_standard_deviation_magn_r',   # cadence removal
                         #'feature_chi2_magn_r',
                         'feature_skew_magn_r', 
                         'feature_standard_deviation_magn_r',
                         'feature_stetson_k_magn_r',
                         'feature_weighted_mean_magn_r',
                         'feature_anderson_darling_normal_flux_r', 
                         'feature_cusum_flux_r',
                         #'feature_eta_e_flux_r', 
                         'feature_excess_variance_flux_r',
                         'feature_kurtosis_flux_r', 
                         'feature_mean_variance_flux_r',
                         #'feature_chi2_flux_r', 
                         'feature_skew_flux_r',
                         'feature_stetson_k_flux_r',

                         'feature_amplitude_magn_g', 
                         'feature_anderson_darling_normal_magn_g',
                         'feature_beyond_1_std_magn_g', 
                         'feature_beyond_2_std_magn_g',
                         'feature_cusum_magn_g', 
                         #'feature_eta_e_magn_g',
                         'feature_inter_percentile_range_2_magn_g',
                         'feature_inter_percentile_range_10_magn_g',
                         'feature_inter_percentile_range_25_magn_g', 
                         'feature_kurtosis_magn_g',
                         'feature_linear_fit_slope_magn_g',
                         'feature_linear_fit_slope_sigma_magn_g',
                         #'feature_linear_fit_reduced_chi2_magn_g', 
                         #'feature_linear_trend_magn_g', # cadence removal
                         #'feature_linear_trend_sigma_magn_g',  # cadence removal
                         'feature_magnitude_percentage_ratio_40_5_magn_g',
                         'feature_magnitude_percentage_ratio_20_5_magn_g',
                         #'feature_maximum_slope_magn_g', 
                         'feature_mean_magn_g',
                         'feature_median_absolute_deviation_magn_g',
                         'feature_median_buffer_range_percentage_10_magn_g',
                         'feature_median_buffer_range_percentage_20_magn_g',
                         'feature_percent_amplitude_magn_g',
                         'feature_percent_difference_magnitude_percentile_5_magn_g',
                         'feature_percent_difference_magnitude_percentile_10_magn_g',
                         #'feature_period_0_magn_g',  # should be negated
                         #'feature_period_s_to_n_0_magn_g', # cadence removal
                         #'feature_period_1_magn_g', 
                         #'feature_period_s_to_n_1_magn_g', # cadence removal
                         #'feature_period_2_magn_g', 
                         #'feature_period_s_to_n_2_magn_g', # cadence removal
                         #'feature_period_3_magn_g', 
                         #'feature_period_s_to_n_3_magn_g', # cadence removal
                         #'feature_period_4_magn_g', 
                         #'feature_period_s_to_n_4_magn_g', # cadence removal
                         #'feature_periodogram_amplitude_magn_g',
                         #'feature_periodogram_beyond_2_std_magn_g',  # cadence removal
                         #'feature_periodogram_beyond_3_std_magn_g', # cadence removal
                         #'feature_periodogram_standard_deviation_magn_g',  # cadence removal
                         #'feature_chi2_magn_g',
                         'feature_skew_magn_g', 
                         'feature_standard_deviation_magn_g',
                         'feature_stetson_k_magn_g', 
                         'feature_weighted_mean_magn_g',
                         'feature_anderson_darling_normal_flux_g', 
                         'feature_cusum_flux_g',
                         #'feature_eta_e_flux_g', 
                         'feature_excess_variance_flux_g',
                         'feature_kurtosis_flux_g', 
                         'feature_mean_variance_flux_g',
                         #'feature_chi2_flux_g', 
                         'feature_skew_flux_g',
                         'feature_stetson_k_flux_g']    

            # Make dictionary. Make keys ZTF IDs
            locus_dict = {}
            ztf_id = locus.properties['ztf_object_id']
            # Get extracted LC feature values from ANTARES db
            locus_dict[ztf_id] = [locus.properties[feature] for feature in feature_names_r_g]
            loci_df = pd.DataFrame.from_dict(locus_dict).T
            loci_df.columns = feature_names_r_g

            # Drop row if any feature values are nan
            loci_df = loci_df[~loci_df[feature_names_r_g].isnull().any(axis=1)]
            
            if len(loci_df) < 1:
                print("Had NaN features. Must drop. Skip!")
                return
            
            #print(loci_df)
            
            try:
                probabilities = alerce.query_probabilities(ztf_id)
                stamp_cls = get_alerce_class(probabilities)
                print("stamp_cls", stamp_cls)
            except:
                print("error getting stamp classifier. Skip!")
                stamp_cls = 'Error'
                return
                
            # TAG Locus!
            locus.tag('LAISS_RFC_AD_filter')
                    

        else:
            locus.properties['anomaly_type'] = 'TBD'
            pass

In [68]:
report = dk.run_filter(LAISS_RFC_AD_filter, locus='ANT2020aes3osq')
#report = dk.run_filter(LAISS_RFC_AD_filter)
print(report)

2023-06-27 14:20:24,476 - WARNING MainThread cassandra.py:list_by_location:1187 - Catalog object not found (catalog_id=19, object_id=898130006)
stamp_cls SLSN
{'locus_id': 'ANT2020aes3osq', 'locus_data': FilterContext(locus_id="ANT2020aes3osq"), 't': 0.008967916000003129, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': {'LAISS_RFC_AD_filter'}, 'raised_halt': False}
